In [1]:
from fastai.text import * 
from fastai.callbacks import *
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle as pkl

torch.cuda.set_device(0)

In [2]:
import unicodedata
import pandas as pd
import re
import spacy
import json

In [3]:
PATH = Path('.')

In [4]:
def normalize_title(title):
    s = re.sub(r'[^a-zA-Z0-9ñç% ]', ' ', unicodedata.normalize('NFKD', title.lower()).encode('ascii', 'ignore').decode("utf-8"))
#    s = re.sub(r'[\d]+', "1", s)
    s = re.sub(r's |s$', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = re.sub(r'(1 )+', '1 ', s)
    return re.sub(r'o |o$', 'a ', s).strip()

In [5]:
with open("main_categories.json", "r") as f:
    cats = json.load(f)

In [6]:
df = pd.read_csv(PATH / 'test.csv')

In [7]:
d = df[df.language == 'portuguese']

In [8]:
d.title = d.title.apply(normalize_title)

/home/jdg/miniconda3/envs/fastai/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
config = awd_lstm_clas_config.copy()
config['qrnn'] = True
config['n_hid'] = 1550 #default 1152
config['n_layers'] = 4 #default 3

In [30]:
processor = SPProcessor.load('',tmp_dir='lm_por_v4')

In [31]:
learn_c = load_learner('/data/anime/ml/', 'classifier_por_v4_3_export', test=d.title)

In [32]:
learn_c.predict('Espelho Veneziano Oval Decorativo Retro Vintage Antigo')

(Category Casa, Moveis e Decoracao,
 tensor(12),
 tensor([2.1547e-02, 1.3029e-02, 5.2541e-03, 5.2216e-03, 7.9526e-03, 1.8766e-02,
         4.9096e-03, 2.6168e-03, 6.4986e-03, 8.1903e-03, 6.9419e-03, 1.5970e-04,
         6.4891e-01, 3.9271e-03, 1.2811e-02, 3.1353e-02, 2.7254e-02, 2.1493e-02,
         3.6358e-02, 1.0258e-03, 8.5602e-03, 9.8066e-04, 6.9761e-03, 6.8136e-02,
         1.5370e-02, 7.8588e-03, 3.2253e-03, 4.6769e-03]))

In [13]:
learn_c.data.show_batch(ds_type=DatasetType.Test, rows=10)

text,target
▁xxbos ▁tw 9 uc 3 r l ci bc cm v 3 ie h v b w ug q n j ld 2 luz y bt d x b w ▁ambar ▁garrafa ▁de,Acessorios para Veiculos
▁xxbos ▁bateria ▁asu ▁k 43 u ▁k 43 sj ▁k 43 s v ▁k 53 b ▁k 53 by ▁k 53 e ▁k 53 f ▁k 53 j ▁k 53,Acessorios para Veiculos
▁xxbos ▁bateria ▁np 500 p 4 c ▁ad 1 br ▁ad 2 br ▁ad 3 br ▁np 550 p 5 c ▁ad 1 br ▁ad 2 br ▁ae 1 b,Acessorios para Veiculos
▁xxbos ▁bateria ▁dell ▁vostra ▁ v 13 ▁ v 130 ▁ v 1300 ▁ v 13 z ▁04 49 tx ▁0 nt g 4 j ▁t 1 g 6 p,Acessorios para Veiculos
▁xxbos ▁bateria ▁asu ▁k 43 b ▁k 43 by ▁k 43 e ▁k 43 f ▁k 43 j ▁k 43 ▁k 43 sj ▁k 43 s v ▁k 43 u,Acessorios para Veiculos
▁xxbos ▁bateria ▁dell ▁p 17 e ▁p 28 f ▁p 37 g ▁p 40 f ▁p 53 g ▁mr 90 y ▁3 421 ▁g 019 y ▁14 ▁8 v,Acessorios para Veiculos
▁xxbos ▁teclada ▁acer ▁e 1 ▁a ez q z 601 010 ▁z ▁n 3 l 82 ▁d 1 b ▁a eq z 600 110 ▁z q z ▁br ▁c,Acessorios para Veiculos
▁xxbos ▁lente ▁yongnua ▁50 mm ▁yn ▁f 1 ▁8 ▁canon ▁70 d ▁t 6 i ▁7 d ▁5 d ▁t 5 i ▁7 d ▁6 d ▁t 3 i,Acessorios para Veiculos
▁xxbos ▁teclada ▁acer ▁p k 130 c 92 a 25 ▁p k 130 c 93 a 25 ▁p k 130 c 94 a 25 ▁5 740 ▁58 10 ▁c,Acessorios para Veiculos
▁xxbos ▁teclada ▁acer ▁ v 5 ▁ 472 ▁6 ▁br 826 ▁ v 5 ▁ 431 ▁ v 5 ▁ 471 ▁mp ▁11 f 76 pa ▁4 422 ▁br,Acessorios para Veiculos


In [14]:
probs = learn_c.get_preds(ds_type=DatasetType.Test, ordered=True)
pred_indices = torch.argmax(probs[0], dim=1)
preds_clas = [learn_c.data.classes[pred] for pred in pred_indices]

In [15]:
learn_c.predict('Kit Maternidade Bolsa-mala Baby/bebe Vinho Menina Kb2012')

(Category Bebes,
 tensor(6),
 tensor([1.8512e-04, 7.2365e-04, 2.7802e-04, 3.3792e-04, 7.0403e-05, 2.6059e-03,
         9.7006e-01, 4.7331e-03, 2.1817e-03, 1.2976e-02, 2.8844e-04, 8.6983e-06,
         1.1978e-03, 1.0801e-04, 1.6061e-04, 2.3147e-04, 7.0266e-04, 4.7184e-04,
         1.0991e-03, 3.8454e-05, 1.1170e-04, 4.6863e-05, 1.1915e-04, 6.5400e-05,
         9.7976e-05, 2.2262e-04, 2.7752e-05, 8.4527e-04]))

In [16]:
probs[0][0]

tensor([8.4734e-05, 5.3279e-04, 1.3279e-04, 1.9145e-05, 1.0313e-05, 6.5441e-04,
        9.8840e-01, 4.0189e-04, 2.3794e-04, 8.3081e-03, 1.8368e-05, 1.2278e-06,
        3.1116e-04, 4.3357e-05, 2.8517e-05, 6.3742e-05, 1.2342e-04, 1.1039e-04,
        2.6298e-04, 1.0569e-05, 3.3712e-05, 1.3677e-05, 2.2649e-05, 1.8131e-05,
        1.8214e-05, 3.8575e-05, 4.8017e-06, 9.4040e-05])

In [17]:
df.loc[df.language == 'portuguese', 'main_category'] = preds_clas

In [18]:
for cat in set(cats.values()):
    learn_c = load_learner('/data/anime/ml/por/', cat + '_model_export', test = df[(df.language == 'portuguese') & (df.main_category == cat)])
    probs = learn_c.get_preds(ds_type=DatasetType.Test, ordered=True)
    pred_indices = torch.argmax(probs[0], dim=1)
    preds_clas = [learn_c.data.classes[pred] for pred in pred_indices]
    df.loc[(df.language == 'portuguese') & (df.main_category == cat), 'category'] = preds_clas

In [19]:
df

,id,title,language,main_category,category
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese,Bebes,DIAPER_BAGS
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese,Bebes,BABY_CHANGING_PADS
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese,Acessorios para Veiculos,ENGINE_COOLING_FAN_MOTORS
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese,Acessorios para Veiculos,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese,Bebes,BABY_CAR_SEATS
5,5,Cabo Freio Mao Tras Direito Vw Up Cod 1s060972...,portuguese,Acessorios para Veiculos,CAR_SCANNERS
6,6,Mini Pc Dell Optiplex Fx160 Atom 2gb Ram Ssd 1...,portuguese,Informatica,MINI_PCS
7,7,Kit Bi Xenon Lâmpada H4 8000k,portuguese,Acessorios para Veiculos,XENON_KITS
8,8,Protetor Pé Botinha-kickboxing Karate Taekwond...,portuguese,Esportes e Fitness,MARTIAL_ARTS_AND_BOXING_GLOVES
9,9,Disco Rigido Externo Western Digital Elements ...,spanish,NaN,NaN


In [20]:
d = df[df.language == 'spanish']

In [21]:
d.title = d.title.apply(normalize_title)

/home/jdg/miniconda3/envs/fastai/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
processor = SPProcessor.load('',tmp_dir='lm_es_v4')

In [23]:
learn_c = load_learner('/data/anime/ml/', 'classifier_es_v4_3_export', test=d.title)

In [24]:
learn_c.data.show_batch(ds_type=DatasetType.Test, rows=10)

text,target
▁xxbos ▁r 4 y co h kl sn x sc f hp v 80 q lt p mp tz f k x a 4 k r lar l jon u v y dr 14 ty y jp r x z sf sr,Acessorios para Veiculos
▁xxbos ▁ir g p 40 72 d p bf ▁ir g p 40 72 d ▁gp 40 72 d ▁ir g p 40 72 ▁40 72 d ▁igbt ▁300 v ▁70 a,Acessorios para Veiculos
▁xxbos ▁igbt ▁ ix gh 60 n 60 c 2 ▁ ix gh ▁60 n 60 c 2 ▁60 n 60 ▁c 2 ▁600 v ▁75 a ▁480 w ▁ta ▁3 p,Acessorios para Veiculos
▁xxbos ▁bateria ▁np ▁bg 1 ▁para ▁sony ▁dsc ▁ w 30 ▁ w 35 ▁ w 40 ▁ w 50 ▁ w 55 ▁ w 70 ▁ w 80 ▁ w 90,Acessorios para Veiculos
▁xxbos ▁fuente ▁ ple 47 f m n 2 ▁ ple 47 f m n 1 ▁ ple 47 f z p 2 ▁lt 47 dr 940 ▁lt 47 da 940,Acessorios para Veiculos
▁xxbos ▁bateria ▁dell ▁j 1 k nd ▁9 j r 2 h ▁ 383 c w ▁n 4 110 ▁n 50 10 ▁n 5 110 ▁n 70 10 ▁n 7 110,Acessorios para Veiculos
▁xxbos ▁at mel ▁24 c 02 ▁sop 8 ▁24 c 02 c ▁1 ▁ 472 ▁at ▁24 c 02 ▁c ▁at ▁24 c 02 ▁c ▁256 k b ▁x 8,Acessorios para Veiculos
▁xxbos ▁set ▁x ▁2 ▁eeprom ▁at 24 c 32 a ▁at 24 c 32 ▁24 c 32 a ▁24 c 32 ▁2 ▁7 ▁a ▁5 ▁5 v ▁sop 8,Acessorios para Veiculos
▁xxbos ▁pantalla ▁14 ▁0 ▁lp 140 wh 1 ▁tl c 6 ▁lt n 140 at 26 ▁l 01 ▁m 140 n w r 2 ▁r 1 ▁h w 23,Acessorios para Veiculos
▁xxbos ▁teclada ▁acer ▁z h 9 ▁5 32 h ▁d 255 ▁d 260 ▁ze 6 ▁na v 50 ▁pa v 70 ▁z h 6 ▁z h 9 ▁ 521,Acessorios para Veiculos


In [25]:
probs = learn_c.get_preds(ds_type=DatasetType.Test, ordered=True)
pred_indices = torch.argmax(probs[0], dim=1)
preds_clas = [learn_c.data.classes[pred] for pred in pred_indices]

In [26]:
df.loc[df.language == 'spanish', 'main_category'] = preds_clas

In [27]:
for cat in set(cats.values()):
    learn_c = load_learner('/data/anime/ml/es/', cat + '_model_export', test = df[(df.language == 'spanish') & (df.main_category == cat)])
    probs = learn_c.get_preds(ds_type=DatasetType.Test, ordered=True)
    pred_indices = torch.argmax(probs[0], dim=1)
    preds_clas = [learn_c.data.classes[pred] for pred in pred_indices]
    df.loc[(df.language == 'spanish') & (df.main_category == cat), 'category'] = preds_clas

In [28]:
d2 = df.copy()
d2.pop('main_category')
d2.pop('title')
d2.pop('language')
d2.to_csv('submission2.csv', index=False)

In [29]:
df

,id,title,language,main_category,category
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese,Bebes,DIAPER_BAGS
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese,Bebes,BABY_CHANGING_PADS
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese,Acessorios para Veiculos,ENGINE_COOLING_FAN_MOTORS
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese,Acessorios para Veiculos,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese,Bebes,BABY_CAR_SEATS
5,5,Cabo Freio Mao Tras Direito Vw Up Cod 1s060972...,portuguese,Acessorios para Veiculos,CAR_SCANNERS
6,6,Mini Pc Dell Optiplex Fx160 Atom 2gb Ram Ssd 1...,portuguese,Informatica,MINI_PCS
7,7,Kit Bi Xenon Lâmpada H4 8000k,portuguese,Acessorios para Veiculos,XENON_KITS
8,8,Protetor Pé Botinha-kickboxing Karate Taekwond...,portuguese,Esportes e Fitness,MARTIAL_ARTS_AND_BOXING_GLOVES
9,9,Disco Rigido Externo Western Digital Elements ...,spanish,Informatica,HARD_DRIVES_AND_SSDS


In [34]:
df.iloc[49]

id                                                              49
title            Espelho Veneziano Oval Decorativo Retro Vintag...
language                                                portuguese
main_category                             Casa, Moveis e Decoracao
category                                                      BEDS
Name: 49, dtype: object

In [35]:
cat = 'Casa, Moveis e Decoracao'
learn_c = load_learner('/data/anime/ml/por/', cat + '_model_export', test = df[(df.language == 'portuguese') & (df.main_category == cat)])